In [1]:
import numpy as np
import pandas as pd
import re
from collections import Counter

In [2]:
data= pd.read_excel('/Users/dongxujia/Dropbox/DS-Discovery_GenderDiversity/04_Data/raw/jobads.xlsx')

In [3]:
data.shape

(6472, 5)

In [12]:
data.id.nunique()

6472

In [5]:
data.head()

,id,job_title_en,job_requirement_en,job_description_en,job_category
0,1,engineering applications it analyst,bachelor degree in computer science engineerin...,development of new tools and change of existin...,information technology--data processing/hardwa...
1,2,ebusiness manager focus online applications,university degree in business administration b...,you are the central key user for web-based onl...,"project management--planning and organization,..."
2,3,ebusiness manager focus online marketing,university degree in business administration b...,you are resonpisble for visibility and attract...,"marketing and sales--marketing,project managem..."
3,4,specialist software services,academic degree computer science business info...,in your new position you want to be part of a ...,information technology--data processing/hardwa...
4,5,head of technical product management brake sys...,degree in engineering with mechanical engineer...,identification of new and innovative product i...,"marketing and sales--other,project management-..."


### • 1) Classify job titles using a two-factor model
	○ Find the area of the job (e.g. software engineering)
		○ (for the area the first element of the job category might be helpful) and
	○ and the rank of the job (e.g. lead, senior, group lead, team leader, assistant) if applicable. 


### • 2) Classify job titles using the degree requirements
	○ Isolate the type of degree and the number of years of worker experience for each position
	○ Group positions by degree x experience groups 

In [17]:
data.head()

,id,job_title_en,job_requirement_en,job_description_en,job_category
0,1,engineering applications it analyst,bachelor degree in computer science engineerin...,development of new tools and change of existin...,information technology--data processing/hardwa...
1,2,ebusiness manager focus online applications,university degree in business administration b...,you are the central key user for web-based onl...,"project management--planning and organization,..."
2,3,ebusiness manager focus online marketing,university degree in business administration b...,you are resonpisble for visibility and attract...,"marketing and sales--marketing,project managem..."
3,4,specialist software services,academic degree computer science business info...,in your new position you want to be part of a ...,information technology--data processing/hardwa...
4,5,head of technical product management brake sys...,degree in engineering with mechanical engineer...,identification of new and innovative product i...,"marketing and sales--other,project management-..."


### Use job_category to extract job_area

In [60]:
# some entries in job_category are null types
data.job_category.isnull().value_counts()

False    6440
True       32
Name: job_category, dtype: int64

In [66]:
# change these to None
data.loc[data.job_category.isnull(),['job_category']] = None

In [74]:
data.job_category.str.contains("--", na=False).value_counts()

True     6415
False      57
Name: job_category, dtype: int64

In [88]:
# note that job_categories that are without "--" are either None or "general management"

data[~data.job_category.str.contains("--", na=False)].head()

,id,job_title_en,job_requirement_en,job_description_en,job_category
28,29,control sys engineer_fix_stpm_1,None,perform system projects for technical concepts...,None
40,41,control sys engineer_fix_stpm_2,None,perform system projects for technical concepts...,None
50,51,control software engineer_ersatz_michael herold,None,Perform software projects for technical concep...,None
701,702,web edi coordinator,None,activity description:\nimplementation of the t...,None
742,743,head of product management workshop solutions,degree in engineering economics or similar qua...,management of the department product managemen...,general management


In [ ]:
# Create a new column called "job_area" that keeps the word before "--" in job_category.
# For entries in job_category that do not contain "--" (either None or "general management"),
# fill in None for the former and "management" for the latter.

In [102]:
data.loc[data.job_category.str.contains(
    "--", na=False),'job_area'] = data.job_category.str.split("--").str[0]

In [96]:
data.loc[data.job_category.isnull(),'job_area'] = None

In [97]:
data.loc[data.job_category == "general management",'job_area'] = "management"

In [116]:
data.job_area = data.job_area.str.replace(","," ")

In [123]:
# create another column job_area_oneword that combine each entry in job_area as one word using underscores

In [124]:
data['job_area_oneword'] = data.job_area.str.replace(" ","_")

In [125]:
data.head()

,id,job_title_en,job_requirement_en,job_description_en,job_category,job_area,job_area_oneword
0,1,engineering applications it analyst,bachelor degree in computer science engineerin...,development of new tools and change of existin...,information technology--data processing/hardwa...,information technology,information_technology
1,2,ebusiness manager focus online applications,university degree in business administration b...,you are the central key user for web-based onl...,"project management--planning and organization,...",project management,project_management
2,3,ebusiness manager focus online marketing,university degree in business administration b...,you are resonpisble for visibility and attract...,"marketing and sales--marketing,project managem...",marketing and sales,marketing_and_sales
3,4,specialist software services,academic degree computer science business info...,in your new position you want to be part of a ...,information technology--data processing/hardwa...,information technology,information_technology
4,5,head of technical product management brake sys...,degree in engineering with mechanical engineer...,identification of new and innovative product i...,"marketing and sales--other,project management-...",marketing and sales,marketing_and_sales


In [128]:
job_area_oneword_counter = Counter(data.job_area_oneword.astype(str))

In [134]:
# 20 most common job_area

job_area_oneword_counter.most_common(20)

[('engineering', 1865),
 ('research_and_development', 931),
 ('quality', 443),
 ('marketing_and_sales', 439),
 ('information_technology', 373),
 ('finance_and_controlling', 373),
 ('project_management', 317),
 ('human_resources', 291),
 ('logistics', 261),
 ('purchasing', 256),
 ('manufacturing_operations_and_production', 208),
 ('industrial_engineering', 183),
 ('administration_and_assistance', 158),
 ('key_account_management', 67),
 ('communications', 62),
 ('None', 32),
 ('law_patents_and_licences', 27),
 ('facility_management', 26),
 ('management', 25),
 ('health_safety_and_security', 24)]

### Extract job_rank (lead, senior, group lead, team leader, assistant)

In [135]:
ranks = ['manager', 'specialist', 'head', 'coach', 'assistant', 
        'expert', 'lead', 'senior', 'junior', 'director', 'supervisor']

In [ ]:
# create column job_rank that extract ranks from job_title_en if job_title_en contain any word in the ranks list
# if no matching word from ranks list is found, then the job_rank should be None

In [136]:
data['job_rank'] = None

In [139]:
for i in ranks:
    data.loc[data.job_title_en.str.contains(i,na=False),'job_rank'] = i

In [142]:
data.job_rank.isnull().value_counts()

True     3297
False    3175
Name: job_rank, dtype: int64

In [145]:
data.head()

,id,job_title_en,job_requirement_en,job_description_en,job_category,job_area,job_area_oneword,job_rank
0,1,engineering applications it analyst,bachelor degree in computer science engineerin...,development of new tools and change of existin...,information technology--data processing/hardwa...,information technology,information_technology,None
1,2,ebusiness manager focus online applications,university degree in business administration b...,you are the central key user for web-based onl...,"project management--planning and organization,...",project management,project_management,manager
2,3,ebusiness manager focus online marketing,university degree in business administration b...,you are resonpisble for visibility and attract...,"marketing and sales--marketing,project managem...",marketing and sales,marketing_and_sales,manager
3,4,specialist software services,academic degree computer science business info...,in your new position you want to be part of a ...,information technology--data processing/hardwa...,information technology,information_technology,specialist
4,5,head of technical product management brake sys...,degree in engineering with mechanical engineer...,identification of new and innovative product i...,"marketing and sales--other,project management-...",marketing and sales,marketing_and_sales,head


### Extract degree and experience information from job_requirement_en

In [ ]:
# job_requirement_en column contains null type values

In [15]:
data.job_requirement_en.isnull().value_counts()

False    6457
True       15
Name: job_requirement_en, dtype: int64

In [50]:
# job_requirement_en column contains entries that only one or two words, such as
# - "tbd", "missing", "x" , "dd", "ot", 
# "xxx","vv","xx","s","bb" "d", "cf epr", "see above"

In [53]:
data[data.job_requirement_en.str.split(" ").str.len() < 3].shape[0]

405

In [16]:
# replacing these entries with None

In [52]:
data.loc[data.job_requirement_en.isnull(),'job_requirement_en'] = None

In [54]:
data.loc[data.job_requirement_en.str.split(" ").str.len() < 3,'job_requirement_en'] = None

In [55]:
data.job_requirement_en.isnull().value_counts()

False    6052
True      420
Name: job_requirement_en, dtype: int64

In [160]:
degrees = ['bachelor','bachelors','university','master','doctor','doctorate','doctoral']

In [161]:
# create column job_degree that extract degree requirements from job_requirement_en 
# if job_requirement_en contain any word in the degrees list
# if no matching word from degrees list is found, then the job_degree should be None

In [162]:
data['job_degree'] = None

In [163]:
for c in degrees:
    data.loc[data.job_requirement_en.str.contains(c,na=False),'job_degree'] = c

In [165]:
data.job_degree.isnull().value_counts()

True     5192
False    1280
Name: job_degree, dtype: int64

In [184]:
data.head()

,id,job_title_en,job_requirement_en,job_description_en,job_category,job_area,job_area_oneword,job_rank,job_degree
0,1,engineering applications it analyst,bachelor degree in computer science engineerin...,development of new tools and change of existin...,information technology--data processing/hardwa...,information technology,information_technology,None,bachelor
1,2,ebusiness manager focus online applications,university degree in business administration b...,you are the central key user for web-based onl...,"project management--planning and organization,...",project management,project_management,manager,university
2,3,ebusiness manager focus online marketing,university degree in business administration b...,you are resonpisble for visibility and attract...,"marketing and sales--marketing,project managem...",marketing and sales,marketing_and_sales,manager,university
3,4,specialist software services,academic degree computer science business info...,in your new position you want to be part of a ...,information technology--data processing/hardwa...,information technology,information_technology,specialist,None
4,5,head of technical product management brake sys...,degree in engineering with mechanical engineer...,identification of new and innovative product i...,"marketing and sales--other,project management-...",marketing and sales,marketing_and_sales,head,master
